In [1]:
from agent_framework import ChatAgent
from agent_framework.observability import setup_observability
from agent_framework.openai import OpenAIChatClient

# Zero-code setup via environment variables
# Set ENABLE_OTEL=true
# Set OTLP_ENDPOINT=http://localhost:4317

# Or manual setup
setup_observability(
    otlp_endpoint="http://localhost:4317"
)

# Create client for the example
client = OpenAIChatClient(model_id="gpt-5")

async def observability_example():
    # Observability is automatically applied to all agents and workflows
    agent = ChatAgent(name="assistant", chat_client=client)
    result = await agent.run("Hello")  # Automatically traced

await observability_example()

# Agent Framework Observability - Service Names

## ✅ Your Setup is Working!
Cell 1 with `setup_observability()` automatically traces all agent operations.

## 🔍 Finding Your Traces in Grafana

### Default Service Name
The agent framework typically uses: **`"unknown_service"`** or **`"python"`**

### TraceQL Queries to Find Your Agent Traces:

1. **Find ALL traces (from any service)**:
```traceql
{}
```

2. **Find traces from Python/agent service**:
```traceql
{ resource.service.name =~ ".*python.*|.*unknown.*" }
```

3. **Find agent operations by span name**:
```traceql
{ name =~ ".*agent.*|.*run.*|.*chat.*" }
```

4. **Find recent traces (last 5 minutes)**:
```traceql
{} | start >= now-5m
```

## 🎯 Customizing Service Name

You can set a custom service name by updating cell 1:

```python
from agent_framework.observability import setup_observability

setup_observability(
    otlp_endpoint="http://localhost:4317",
    service_name="my-agent-app"  # Custom name
)
```

Then query with:
```traceql
{ resource.service.name = "my-agent-app" }
```

## 📊 View in Grafana

1. Open http://localhost:3000
2. Go to **Explore** → Select **Tempo**
3. Start with the simple query: `{}`
4. Look for your service in the results!


In [2]:
from agent_framework import ChatAgent, ai_function
from agent_framework.openai import OpenAIChatClient

# Create simple tools for the example
@ai_function
def get_weather(location: str) -> str:
    """Get weather for a location."""
    return f"Weather in {location}: sunny"

@ai_function
def get_time() -> str:
    """Get current time."""
    return "Current time: 2:30 PM"

# Create client
client = OpenAIChatClient(model_id="gpt-5")

async def example():
    # Direct creation
    agent = ChatAgent(
        name="assistant",
        chat_client=client,
        instructions="You are a helpful assistant.",
        tools=[get_weather]  # Multi-turn by default
    )

    # Factory method (more convenient)
    agent = client.create_agent(
        name="assistant",
        instructions="You are a helpful assistant.",
        tools=[get_weather]
    )

    # Execution with runtime tool configuration
    result = await agent.run(
        "What's the weather in Bangkok?",
        tools=[get_time],  # Can add tools at runtime
        tool_choice="auto"
    )
    print(result)

await example()

It’s sunny in Bangkok. Want the temperature or a short forecast too?


In [1]:
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

exporter = OTLPSpanExporter(endpoint="localhost:4317", insecure=True)
provider = TracerProvider()
provider.add_span_processor(BatchSpanProcessor(exporter))
# configure global provider

# TraceQL Queries for Grafana Tempo

## How to Use These Queries:
1. Open Grafana at http://localhost:3000
2. Go to **Explore** (compass icon on left)
3. Select **Tempo** as the data source
4. Paste these queries in the TraceQL editor

---

## 🎯 Quick Start Queries for Agent Framework

### 1. Find ALL traces (easiest way to start!)
```traceql
{}
```

### 2. Find agent framework traces (unknown service name)
```traceql
{ resource.service.name =~ ".*unknown.*|.*python.*" }
```

### 3. Find recent traces (last 5 minutes)
```traceql
{} | start >= now-5m
```

### 4. Find traces by agent operations
```traceql
{ name =~ ".*agent.*|.*run.*|.*chat.*" }
```

---

## Basic TraceQL Queries

### 5. Find all traces from your service
```traceql
{ resource.service.name = "agent-framework-demo" }
```

### 6. Find traces by span name
```traceql
{ name = "agent_workflow" }
```

### 7. Find traces with specific attributes
```traceql
{ span.workflow.type = "sequential" }
```

### 8. Find traces by environment
```traceql
{ resource.service.name = "agent-framework-demo" && span.environment = "development" }
```

### 9. Find traces with specific task types
```traceql
{ span.task = "review" }
```

---

## Advanced TraceQL Queries

### 10. Find traces with duration > 100ms
```traceql
{ resource.service.name = "agent-framework-demo" } | duration > 100ms
```

### 11. Find traces with errors
```traceql
{ status = error }
```

### 12. Find traces with specific span attributes
```traceql
{ span.decision = "approve" }
```

### 13. Combine multiple conditions (AND)
```traceql
{ resource.service.name = "agent-framework-demo" && span.task = "review" && span.decision = "approve" }
```

### 14. Find traces by span count
```traceql
{ resource.service.name = "agent-framework-demo" } | spanCount > 3
```

---

## Aggregation Queries

### 15. Count traces by service
```traceql
{ } | by(resource.service.name) | count() > 0
```

### 16. Average duration by task type
```traceql
{ resource.service.name = "agent-framework-demo" } | by(span.task) | avg(duration)
```

### 17. P95 duration
```traceql
{ resource.service.name = "agent-framework-demo" } | quantile_over_time(duration, 0.95)
```

---

## Time-based Queries

### 18. Traces in the last 15 minutes
```traceql
{} | start >= now-15m
```

### 19. Traces in the last hour
```traceql
{} | start >= now-1h
```

### 20. Traces between specific times
```traceql
{ resource.service.name = "agent-framework-demo" } && (start >= 2025-11-02T00:00:00Z && start <= 2025-11-02T23:59:59Z)
```

---

## Pattern Matching

### 21. Find spans with names containing "task"
```traceql
{ name =~ ".*task.*" }
```

### 22. Find traces with any attribute matching a pattern
```traceql
{ span.task =~ ".*review.*" }
```

---

## Working with Events

### 23. Find traces with specific events
```traceql
{ resource.service.name = "agent-framework-demo" && span.event.name = "Draft completed" }
```

---

## Practical Examples for Your Use Case

### 24. Find all agent workflow traces
```traceql
{ name = "agent_workflow" && resource.service.name = "agent-framework-demo" }
```

### 25. Find slow workflows (> 500ms)
```traceql
{ name = "agent_workflow" } | duration > 500ms
```

### 26. Find all writer tasks
```traceql
{ name = "writer_task" }
```

### 27. Find approved reviews
```traceql
{ span.task = "review" && span.decision = "approve" }
```

---

## Tips:
- ⭐ **Start with `{}` to see ALL traces** - this is the easiest way!
- Use `&&` for AND conditions
- Use `||` for OR conditions  
- Use `=~` for regex matching
- Use `!=` for not equals
- Attributes are accessed with `span.attribute_name` or `resource.attribute_name`
- Duration filters use suffixes: `ms`, `s`, `m`, `h`
- Time filters use `now-5m`, `now-1h`, `now-1d` format

# LogQL Queries for Grafana Loki (Logs)

## How to Use These Queries:
1. Open Grafana at http://localhost:3000
2. Go to **Explore** (compass icon on left)
3. Select **Loki** as the data source
4. Paste these queries in the LogQL editor

---

## Basic LogQL Queries

### 1. Get all logs
```logql
{job="varlogs"}
```

### 2. Get logs from specific service (if you're sending logs with service name)
```logql
{service_name="agent-framework-demo"}
```

### 3. Get logs by level
```logql
{service_name="agent-framework-demo"} | level = "error"
```

### 4. Filter logs containing specific text
```logql
{service_name="agent-framework-demo"} |= "workflow"
```

### 5. Exclude logs containing text
```logql
{service_name="agent-framework-demo"} != "debug"
```

---

## Advanced LogQL Queries

### 6. Regex filter
```logql
{service_name="agent-framework-demo"} |~ "error|warning"
```

### 7. Case-insensitive search
```logql
{service_name="agent-framework-demo"} |~ `(?i)error`
```

### 8. Parse JSON logs
```logql
{service_name="agent-framework-demo"} | json | level="error"
```

### 9. Extract and filter by field
```logql
{service_name="agent-framework-demo"} | json | task="review"
```

### 10. Multiple filters
```logql
{service_name="agent-framework-demo"} |= "task" |= "completed"
```

---

## Aggregation Queries

### 11. Count logs per service
```logql
count_over_time({service_name="agent-framework-demo"}[5m])
```

### 12. Rate of logs
```logql
rate({service_name="agent-framework-demo"}[1m])
```

### 13. Count errors in last hour
```logql
sum(count_over_time({service_name="agent-framework-demo"} |= "error" [1h]))
```

### 14. Top 10 log patterns
```logql
topk(10, sum by (level) (count_over_time({service_name="agent-framework-demo"}[1h])))
```

---

## Correlation with Traces

### 15. Find logs for a specific trace ID
```logql
{service_name="agent-framework-demo"} | json | trace_id="your-trace-id-here"
```

### 16. Find logs with trace context
```logql
{service_name="agent-framework-demo"} | json | trace_id != ""
```

---

## Note on Logs:
To send logs to Loki, you need to configure OpenTelemetry logging:

```python
from opentelemetry import _logs
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import OTLPLogExporter

# Set up logging
logger_provider = LoggerProvider()
logger_provider.add_log_record_processor(
    BatchLogRecordProcessor(OTLPLogExporter(endpoint="localhost:4317", insecure=True))
)
_logs.set_logger_provider(logger_provider)
```

# 🎯 Complete Setup Summary - What Made It Work

## Service Name: `agent_framework`

---

## 📋 Problems We Solved (In Order)

### 1. **Deprecated Logging Exporter** ❌→✅
**Problem:** Collector failed to start with error:
```
'exporters' the logging exporter has been deprecated, use the debug exporter instead
```

**Solution:** Updated `collector-config.yaml`:
```yaml
exporters:
  debug: {}  # Changed from logging: {}
service:
  pipelines:
    traces:
      exporters: [debug]  # Changed from [logging]
```

---

### 2. **YAML Structure Error** ❌→✅
**Problem:** Wrong YAML indentation - exporters were nested under wrong parent

**Solution:** Fixed structure in `collector-config.yaml`:
```yaml
# WRONG:
metrics:
  exporters: [debug]

# CORRECT:
service:
  pipelines:
    metrics:
      exporters: [debug]
```

---

### 3. **Docker Port Mapping Not Working** ❌→✅
**Problem:** Python app couldn't connect to collector at `localhost:4317` (StatusCode.UNAVAILABLE)

**Solution:** Changed receiver binding in `collector-config.yaml`:
```yaml
receivers:
  otlp:
    protocols:
      grpc:
        endpoint: 0.0.0.0:4317  # Changed from 127.0.0.1:4317
      http:
        endpoint: 0.0.0.0:4318
```

**Why:** Docker port publishing (-p 4317:4317) requires binding to 0.0.0.0, not 127.0.0.1

---

### 4. **Port 4317 Already in Use** ❌→✅
**Problem:** 
```
Bind for 0.0.0.0:4317 failed: port is already allocated
```

**Solution:** 
```powershell
docker system prune -f
```
Cleared orphaned containers holding the port.

---

### 5. **UNIMPLEMENTED Error for Traces** ❌→✅
**Problem:** Python app got `StatusCode.UNIMPLEMENTED` when sending traces

**Solution:** Added missing pipelines to `collector-config.yaml`:
```yaml
service:
  pipelines:
    traces:      # Was missing!
      receivers: [otlp]
      exporters: [debug]
    logs:        # Was missing!
      receivers: [otlp]
      exporters: [debug]
    metrics:     # Already existed
      receivers: [otlp]
      exporters: [debug]
```

**Why:** Collector only had metrics pipeline, couldn't handle trace/log data.

---

### 6. **Trace Visualization Needed** ❌→✅
**Problem:** Traces were being collected but nowhere to visualize them

**Solution:** Set up complete Grafana stack with `docker-compose`:

**Created `grafana-stack-compose.yaml`:**
- **Tempo** - Trace storage and querying
- **Loki** - Log aggregation
- **Prometheus** - Metrics storage
- **Grafana** - Unified visualization UI

---

### 7. **Tempo Permission Error** ❌→✅
**Problem:** Tempo container failed:
```
failed to create store: mkdir /tmp/tempo/blocks: permission denied
```

**Solution:** Fixed in `tempo-config.yaml` and `docker-compose`:
```yaml
# tempo-config.yaml
storage:
  trace:
    backend: local
    wal:
      path: /var/tempo/wal      # Changed from /tmp/tempo
    local:
      path: /var/tempo/blocks

# docker-compose
services:
  tempo:
    user: root                   # Added
    volumes:
      - tempo-data:/var/tempo   # Changed mount point
```

---

### 8. **Collector Can't Reach Tempo** ❌→✅
**Problem:** Collector and Tempo in different Docker networks

**Solution:** 
1. Started both on same Docker network:
```powershell
docker run --name otelcol \
  --network learning_microsoft_agent_framework_grafana-net \
  -p 4317:4317 -p 4318:4318 \
  -v "${PWD}\collector-config.yaml:/etc/otel/config.yaml" \
  otel/opentelemetry-collector:latest
```

2. Updated `collector-config.yaml` to use service names:
```yaml
exporters:
  otlphttp/tempo:
    endpoint: http://tempo:4318      # Service name, not IP
  otlphttp/loki:
    endpoint: http://loki:3100/otlp
  prometheusremotewrite:
    endpoint: http://prometheus:9090/api/v1/write

service:
  pipelines:
    traces:
      receivers: [otlp]
      exporters: [debug, otlphttp/tempo]
    logs:
      receivers: [otlp]
      exporters: [debug, otlphttp/loki]
    metrics:
      receivers: [otlp]
      exporters: [debug, prometheusremotewrite]
```

---

### 9. **TracerProvider Override Warning** ❌→✅
**Problem:** 
```
WARNING: Overriding of current TracerProvider is not allowed
```

**Solution:** Removed duplicate TracerProvider setup. Used existing provider from `setup_observability()` in cell 1.

---

## 🏗️ Final Architecture

```
┌─────────────────────┐
│   Python App        │
│  (agent_framework)  │
│  Cell 1: setup_     │
│  observability()    │
└──────────┬──────────┘
           │ OTLP gRPC
           │ localhost:4317
           ▼
┌─────────────────────┐
│  OpenTelemetry      │
│  Collector          │
│  - Receives on      │
│    0.0.0.0:4317/18  │
│  - Exports to       │
│    Grafana stack    │
└──────────┬──────────┘
           │ Docker Network
           │ (grafana-net)
           ▼
┌─────────────────────────────────┐
│      Grafana Stack              │
│  ┌────────┬────────┬──────────┐ │
│  │ Tempo  │ Loki   │Prometheus│ │
│  │(traces)│ (logs) │ (metrics)│ │
│  └────┬───┴───┬────┴────┬─────┘ │
│       └───────┼─────────┘       │
│               ▼                 │
│          Grafana UI             │
│       localhost:3000            │
└─────────────────────────────────┘
```

---

## 📁 Key Files Created/Modified

### 1. `collector-config.yaml`
```yaml
receivers:
  otlp:
    protocols:
      grpc:
        endpoint: 0.0.0.0:4317
      http:
        endpoint: 0.0.0.0:4318

exporters:
  debug: {}
  otlphttp/tempo:
    endpoint: http://tempo:4318
    tls:
      insecure: true
  otlphttp/loki:
    endpoint: http://loki:3100/otlp
    tls:
      insecure: true
  prometheusremotewrite:
    endpoint: http://prometheus:9090/api/v1/write
    tls:
      insecure: true

service:
  pipelines:
    traces:
      receivers: [otlp]
      exporters: [debug, otlphttp/tempo]
    metrics:
      receivers: [otlp]
      exporters: [debug, prometheusremotewrite]
    logs:
      receivers: [otlp]
      exporters: [debug, otlphttp/loki]
```

### 2. `grafana-stack-compose.yaml`
- Tempo: Ports 3200 (query), 4317/4318 internal
- Grafana: Port 3000
- Loki: Port 3100
- Prometheus: Port 9090

### 3. `tempo-config.yaml`
- OTLP receivers on 4317/4318
- Storage at /var/tempo

### 4. `grafana-datasources.yaml`
- Auto-configured Tempo, Loki, Prometheus

---

## 🎯 TraceQL Query for Your Service

```traceql
{ resource.service.name = "agent_framework" }
```

Or find recent traces:
```traceql
{ resource.service.name = "agent_framework" } | start >= now-15m
```

---

## ✅ Verification Commands

```powershell
# Check all containers running
docker ps

# View collector logs
docker logs otelcol --tail 50

# View Tempo logs
docker logs learning_microsoft_agent_framework-tempo-1

# Check for traces in collector
docker logs otelcol | Select-String -Pattern "Traces"
```

---

## 🚀 Access Points

- **Grafana UI:** http://localhost:3000
- **Tempo API:** http://localhost:3200
- **Loki API:** http://localhost:3100
- **Prometheus UI:** http://localhost:9090
- **Collector Receiver:** localhost:4317 (gRPC), localhost:4318 (HTTP)

---

## 💡 Key Learnings

1. **Docker networking:** Containers on same network use service names, not localhost
2. **0.0.0.0 binding:** Required for Docker port publishing to work from host
3. **OTLP protocols:** gRPC (4317) vs HTTP (4318) - must match in app and collector
4. **Pipeline configuration:** Collector needs explicit pipelines for traces/logs/metrics
5. **Service names:** Set via `setup_observability(service_name="...")` for easy filtering